In [ ]:
import numpy as np
import copy
import pandas as pd
import open3d as o3d

In [ ]:
def iterative_colored_icp(source, target):
    """ 
    Based on: http://www.open3d.org/docs/release/python_example/pipelines/index.html#colored-pointcloud-registration-py
    """
    # Colored pointcloud registration.
    # This is implementation of following paper:
    # J. Park, Q.-Y. Zhou, V. Koltun,
    # Colored Point Cloud Registration Revisited, ICCV 2017.
    voxel_radiuses = [0.04, 0.02, 0.01]
    max_iterations = [50 * 100, 30, 14]
    current_transformation = np.identity(4)
    
    print("Colored point cloud registration...")
    for i, (voxel_radius, max_iteration) in enumerate(zip(voxel_radiuses, max_iterations)):
        print(f"1. Iteration num {i}")
        print(f"2. Downsampling point clouds, with a voxel size={voxel_radius:.2f}")
        source_down = source.voxel_down_sample(voxel_radius)
        target_down = target.voxel_down_sample(voxel_radius)

        print("2. Estimate normal")
        source_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_radius * 2, max_nn=30))
        target_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_radius * 2, max_nn=30))

        print(f"3. Applying colored point cloud registration, with max_iteration={max_iteration}")
        result_icp = o3d.pipelines.registration.registration_colored_icp(
            source_down,
            target_down,
            voxel_radius,
            current_transformation,
            o3d.pipelines.registration.TransformationEstimationForColoredICP(),
            o3d.pipelines.registration.ICPConvergenceCriteria(
                relative_fitness=1e-6, relative_rmse=1e-6, max_iteration=max_iteration
            ),
        )
        current_transformation = result_icp.transformation
        print(result_icp, "\n")
    return current_transformation


def iterative_point_to_point_icp(source, target):
    """ 
    Based on: http://www.open3d.org/docs/release/python_example/pipelines/index.html#colored-pointcloud-registration-py
    """
    
    # Init
    current_transformation = np.identity(4)
    voxel_radiuses = [0.04, 0.04, 0.02, 0.02, 0.01, 0.01]
    max_iterations = [50, 50, 30, 30, 14, 14]

    # Iterative icp registration
    for i, (voxel_radius, max_iteration) in enumerate(zip(voxel_radiuses, max_iterations)):
        print(f"1. Iteration num {i}")
        print(f"2. Downsampling point clouds, with a voxel size={voxel_radius:.2f}")
        source_down = source.voxel_down_sample(voxel_radius)
        target_down = target.voxel_down_sample(voxel_radius)

        print(f"3. Applying point cloud registration, with max_iteration={max_iteration}")
        result_icp = o3d.pipelines.registration.registration_icp(
            source_down,
            target_down,
            voxel_radius,
            current_transformation,
            o3d.pipelines.registration.TransformationEstimationPointToPoint(),
            o3d.pipelines.registration.ICPConvergenceCriteria(
                relative_fitness=1e-6, relative_rmse=1e-6, max_iteration=max_iteration
            ),
        )
        current_transformation = result_icp.transformation
        print(result_icp, "\n")
        print(current_transformation, "\n")
    return current_transformation


def draw_registration_result(source, target, source_transform=None):
    "Used to visualize a transformation on the source object."
    # Default value for transformation
    source_transform = (np.identity(4) if source_transform is None else source_transform)

    # Show objects
    source_temp = copy.deepcopy(source)
    source_temp.transform(source_transform)
    o3d.visualization.draw_geometries([source_temp, target])


def draw_registration_result_color_coded(source, target, source_transform=None):
    "Used to visualize a transformation on the source object, where the objects are given distinct colors."
    # Default value for transformation
    source_transform = (np.identity(4) if source_transform is None else source_transform)

    # Color and show objects
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([0, 0.706, 1])  # Blue
    target_temp.paint_uniform_color([1, 0.706, 0])  # Yellow
    source_temp.transform(source_transform)
    o3d.visualization.draw_geometries([source_temp, target_temp])

## Present for reference only
# def draw_from_perspective(source, target):
#     o3d.visualization.draw_geometries(
#         [source, target],
#         zoom=0.5,
#         front=[-0.2458, -0.8088, 0.5342],
#         lookat=[1.7745, 2.2305, 0.9787],
#         up=[0.3109, -0.5878, -0.7468],
#     )


In [ ]:
# Steps:
# 1. Download Polycam 3d-scanned .glb files
# 2. Open .glb file in Meshlab (a free and open-source program)
# 3. Export as .obj file, save texture as separate file
# 4. Open new project and import previously exported .obj file
# 5. Apply remeshing > sub-division with Catmull-Clark method
# 6. Apply color creation > Transfer texture to vertex (requires a separate texture file from step 3)
# 7. Apply selection > delete all faces
# 8. Export as .ply file, excluding any face and wedge data in the export wizard
# 9. <the rest of this script>

# Runtime variable
# doColorCoding = True
doColorCoding = False

# Read point clouds containing color data
filename_1 = r"../3D skanningar/Sigrids 1_meshlab.ply"
filename_2 = r"../3D skanningar/Sigrids 2_meshlab.ply"
pointCloud1 = o3d.io.read_point_cloud(filename_1)
pointCloud2 = o3d.io.read_point_cloud(filename_2)

# Rotate second point cloud by arbitrary amount
rotationMatrix = pointCloud2.get_rotation_matrix_from_xyz((np.pi / 4, np.pi / 4, 0)) # /4 works for icp, but /3 does not, too far away from correct orientation?
pointCloud2.rotate(rotationMatrix, center=(0, 0, 0))

# Move second point cloud to avoid having the point clouds overlapp
pointCloud2.translate(np.asarray([0.052, 0, 0]))

# Show the clouds
if (doColorCoding): draw_registration_result_color_coded(pointCloud1, pointCloud2)
else:               draw_registration_result(pointCloud1, pointCloud2)
        
# Execute ICP registration
resulting_transform = iterative_point_to_point_icp(pointCloud1, pointCloud2)
# resulting_transform = iterative_colored_icp(pointCloud1, pointCloud2)

# Show the clouds
if (doColorCoding): draw_registration_result_color_coded(pointCloud1, pointCloud2, resulting_transform)
else:               draw_registration_result(pointCloud1, pointCloud2, resulting_transform)


In [ ]:
# TODO Create interactive multi-window solution
# http://www.open3d.org/docs/release/python_example/visualization/index.html#multiple-windows-py
# http://www.open3d.org/docs/release/python_example/visualization/index.html#non-blocking-visualization-py

# useBlockingVisualization = False
# 
# vis1 = o3d.visualization.Visualizer()
# vis1.create_window()
# vis1.add_geometry(pointCloud1)
# vis1.add_geometry(pointCloud2)
# 
# vis1.poll_events()
# vis1.update_renderer()
# 
# vis2 = o3d.visualization.Visualizer()
# vis2.create_window()
# vis2.add_geometry(copy.deepcopy(pointCloud1).transform(resulting_transform))
# vis2.add_geometry(pointCloud2)
# 
# vis2.poll_events()
# vis2.update_renderer()
# 
# vis1.destroy_window()
# vis2.destroy_window()


In [ ]:
# NOTE Here for reference only

# filename_1 = r"../3D skanningar/Polycam Sigrids 1.glb"
# filename_2 = r"../3D skanningar/Polycam Sigrids 2.glb"

# mesh_stl_1 = o3d.io.read_triangle_mesh(filename_1)
# mesh_stl_2 = o3d.io.read_triangle_mesh(filename_2)
# point_stl_1 = mesh_stl_1.sample_points_uniformly(number_of_points=120000)
# point_stl_2 = mesh_stl_2.sample_points_uniformly(number_of_points=120000)

# print('Vertex Colors:')
# print(np.asarray(mesh_stl_1.vertex_colors))
# print(np.asarray(mesh_stl_2.vertex_colors))
# print("Done")

# o3d.visualization.draw_geometries([mesh_stl_1, mesh_stl_2])
# o3d.visualization.draw_geometries([point_stl_1, point_stl_2])